# PyPI project classification based on package description

This is an experimental notebook with the goal of exploring different methods to extend [project2vec](https://developers.redhat.com/articles/2021/10/06/find-and-compare-python-libraries-project2vec).

The data used for this project consists in information about ~70 000 Python packages aggregated from PyPI using [Selinon](https://github.com/thoth-station/selinon-worker) and is available under ``s3://DH-DEV-DATA/data/thoth/selinon/pypi_project/ProjectInfo/`` on Ceph.

## Data description

Each file describes a different package hosted on PyPI, providing metadata about the project in the `info` field. We are interested in the following subfields:
- `classifiers`
- `keywords`
- `description`

The classifiers of a project consist in a hierchical labels, with their [official list](https://pypi.org/classifiers/) described on the PyPI website. They allow to classify a project according to different categories and scopes, for example by language and version, operating system or topic. In the case of the `selinon` package, the classifiers are the following:

```
      "Development Status :: 4 - Beta",
      "Intended Audience :: Developers",
      "License :: OSI Approved :: BSD License",
      "Operating System :: OS Independent",
      "Programming Language :: Python :: 3",
      "Programming Language :: Python :: 3.4",
      "Programming Language :: Python :: 3.5",
      "Programming Language :: Python :: 3.6",
      "Programming Language :: Python :: Implementation :: CPython",
      "Programming Language :: Python :: Implementation :: PyPy",
      "Topic :: System :: Distributed Computing"
```

The keywords consist in a few words written in free text format (often separated by a whitespace or a coma) describing the package in a non-hierchical way. The keywords chosen to describe `selinon` are:

``selinon celery yaml flow distributed-computing``

Finally, the description is the text describing the project as seen in the PyPI pakage page.


## Creation of a raw content dataset

Let's create datasets of descriptions from those PyPI projects and link each project and its description respectively to its trove classifiers and keywords from PyPI.
The dataset should contain the following information:

``Project name | Project description | Project PyPI classifiers or Project PyPI keywords``

In [159]:
from collections import OrderedDict
import csv
import json
import nltk
from nltk.corpus import stopwords
import os
import pandas as pd
from pprint import PrettyPrinter
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from typing import Dict
from typing import List

In [160]:
_DATA_PATH = "data/"
_DATASETS_SAVING_PATH = "datasets/"

### Assembling and saving the datasets:

Formatting the projects descriptions by removing punctuation is necessary for text preprocessing purposes and to avoid any conflicts when storing data in CSV format:

In [161]:
def format_description(description: str, threshold: int) -> str:
    """Normalize the description and remove special characters, non-English characters and words longer than a threshold."""
    symbols = "!\"#$%&()*+-/:;<=>?@[\]^_`{|}~\n"
    for symbol in symbols:
        description = description.replace(symbol, " ")
    description = " ".join([word.strip().lower() for word in description.split(" ") if word != ""])

    words_list = description.split(" ")
    for word in words_list:
        if len(word) >= threshold:
            words_list.remove(word)
    formatted_description = " ".join(words_list).encode("ascii", "ignore").decode()

    return formatted_description

In [162]:
def format_keywords(keywords: str, threshold: int) -> str:
    """Format free text PyPI keywords of the projects."""
    keywords = str(keywords)
    if "," in keywords:
        return format_description(keywords.replace(",", " "), threshold)
    return keywords

As we need only projects containing a full description, we filter those with an empty or unknown one:

In [163]:
def description_is_valid(description: str) -> bool:
    """Verify if a description is present and valid."""
    if description in [None, "", " "] or description.startswith(("Unknown", "unknown", "UNKNOWN", "\n")):
        return False
    return True

In [164]:
def get_projects_with_descriptors_dataset(descriptors: str, words_length_threshold: int) -> Dict[str, list]:
    """Retrieve projects with PyPI classifiers or keywords from data aggregated with Selinon."""
    projects_with_descriptors_dataset = {}
    for root, dirs, files in os.walk(os.path.join(_DATA_PATH)):
        for file in files:
            with open(os.path.join(_DATA_PATH, file), "r") as json_file:
                file_content = json.loads(json_file.read())
            
                if descriptors == "classifiers":
                    if description_is_valid(file_content["info"]["description"]) and file_content["info"]["classifiers"] != []:
                        project_description = format_description(file_content["info"]["description"], words_length_threshold)
                        projects_with_descriptors_dataset[file] = [project_description, file_content["info"]["classifiers"]]
                elif descriptors == "keywords":
                    if description_is_valid(file_content["info"]["description"]) and file_content["info"]["keywords"] not in ["", None]:
                        project_description = format_description(file_content["info"]["description"], words_length_threshold)
                        projects_with_descriptors_dataset[file] = [project_description, format_keywords(file_content["info"]["keywords"], words_length_threshold)]
                else:
                    raise ValueError("Invalid descriptors name specified")

    return projects_with_descriptors_dataset

In order to test the project, we create a subsample of the datasets by selecting randomly a limited number of packages:

In [165]:
NUMBER_OF_PROJECTS = 100
WORD_LENGTH_THRESHOLD = 25

projects_with_keywords_dataset = get_projects_with_descriptors_dataset("keywords", WORD_LENGTH_THRESHOLD)
projects_with_classifiers_dataset = get_projects_with_descriptors_dataset("classifiers", WORD_LENGTH_THRESHOLD)

random_keys = list(set(list(projects_with_keywords_dataset.keys())).intersection(set(list(projects_with_classifiers_dataset.keys()))))

random.shuffle(random_keys)
random_keys = random_keys[:min(len(random_keys)-1, NUMBER_OF_PROJECTS)]

with open(os.path.join(_DATASETS_SAVING_PATH, f"projects_with_keywords_dataset_{NUMBER_OF_PROJECTS}.csv"), "w") as csv_file:
    writer = csv.writer(csv_file)
    for project_name in random_keys:
        writer.writerow([project_name, projects_with_keywords_dataset[project_name][0], projects_with_keywords_dataset[project_name][1]])

with open(os.path.join(_DATASETS_SAVING_PATH, f"projects_with_classifiers_dataset_{NUMBER_OF_PROJECTS}.csv"), "w") as csv_file:
    writer = csv.writer(csv_file)
    for project_name in random_keys:
        writer.writerow([project_name, projects_with_classifiers_dataset[project_name][0], ",".join(projects_with_classifiers_dataset[project_name][1])])
        

We have now built two datasets containing each project's name and description, followed by corresponding classifiers or keywords.

## Token ranking using TF-IDF 

This step consists in building a dataset that contains tokens of the description for each package ranked by importance based on the [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) tokens ranking method. 

TF-IDF (for Term Frequency - Inverse Document Frequency) is a statistical method to find the most important terms of a document from a document corpus, i.e. a set of documents. The technique relies on an analysis of the frequency of terms in tokenized and pre-processed documents, where the most frequent terms across documents of the corpus are penalized to highlight the terms appearing more frequently in the document, which allows to identify the topic of the analyzed text. The corpus of documents considered in this analysis is the set of text descriptions from PyPI Python packages which are described with at least one keyword on the PyPI index. 

A ranking of tokens with TF-IDF can be used to identify the topic of a package which has not been classified yet by computing a similarity measure between its vectorized version with TF-IDF and specific keywords. 

### Text pre-processing

The first step of the descriptions text preprocessing is to filter them in order to extract only relevant words (removing stopwords).

In [166]:
nltk.download('stopwords')

def preprocess_dataset(dataset_name: str) -> Dict[str, list]:
    """Remove stop words and put tokens in lowercase."""
    preprocessed_dataset = {}
    stopwords_complete_list = stopwords.words('english') + [stopword.capitalize() for stopword in stopwords.words('english')]

    with open(os.path.join(_DATASETS_SAVING_PATH, dataset_name), "r") as csv_file:
        reader = csv.reader(csv_file)
        for row in reader:
            preprocessed_description = list(filter(str.isalnum, row[1].split(" ")))
            preprocessed_description = [word.lower() for word in preprocessed_description if word not in stopwords_complete_list]
            preprocessed_dataset[row[0]] = [preprocessed_description, row[2].split(",")]

    return preprocessed_dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mcostant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [167]:
# Increasing field size limit to avoid size-related errors:
csv.field_size_limit(sys.maxsize)

9223372036854775807

Let's build a dataset containing only the project names and descriptions

In [168]:
preprocessed_projects_with_classifiers_dataset = preprocess_dataset(f"projects_with_classifiers_dataset_{NUMBER_OF_PROJECTS}.csv")

# Print the dataset:

# pp = PrettyPrinter(indent=2)
# pp.pprint(preprocessed_projects_with_classifiers_dataset)

In [169]:
preprocessed_projects_with_keywords_dataset = preprocess_dataset(f"projects_with_keywords_dataset_{NUMBER_OF_PROJECTS}.csv")

# Print the dataset:

# pp = PrettyPrinter(indent=2)
# pp.pprint(preprocessed_projects_with_keywords_dataset)

In [170]:
preprocessed_projects_with_keywords_dataset_ordered = OrderedDict(sorted(preprocessed_projects_with_keywords_dataset.items()))

corpus_keywords = [" ".join(description[0]) for package_name, description in preprocessed_projects_with_keywords_dataset_ordered.items()]

vectorizer = TfidfVectorizer(use_idf=True)
tf_idf_keywords = vectorizer.fit_transform(corpus_keywords)

Let's have a first overview of the results we get from the TF-IDF token ranking by comparing the last obtained rankings for the first 10 packages to the corresponding project descriptions:

In [171]:
packages = list(preprocessed_projects_with_keywords_dataset_ordered.keys())[:10]
package_indexes = OrderedDict({package : index for index, package in enumerate(packages)})

for package_name in packages:
    df_keywords = pd.DataFrame(tf_idf_keywords[package_indexes[package_name]].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"])
    df_keywords = df_keywords.sort_values("TF-IDF", ascending=False)

    print("Package name: ", package_name)
    print("Description: ", projects_with_keywords_dataset[package_name][0])
    print("Highest ranked keywords:\n")
    print(df_keywords.head(20))

    print("\n\n\n")

Package name:  adafruit-circuitpython-amg88xx
Description:  introduction .. image https readthedocs.org projects adafruit circuitpython amg88xx badge version latest target https projects amg88xx en latest alt documentation status .. image https img.shields.io discord 327254708534116352.svg target https discord.gg nbqh6qu alt discord .. image https travis ci.com adafruit adafruit circuitpython amg88xx.svg branch master target https travis ci.com adafruit adafruit circuitpython amg88xx alt build status adafruit circuitpython module for the amg88xx grid eye ir 8x8 thermal camera. dependencies this driver depends on adafruit circuitpython https github.com adafruit circuitpython bus device https github.com adafruit adafruit circuitpython busdevice register https github.com adafruit adafruit circuitpython register please ensure all dependencies are available on the circuitpython filesystem. this is easily achieved by downloading the adafruit library and driver bundle https github.com adafrui

At first glance, we can see that the highest ranked tokens seem to be relevant descriptors of the projects, with the exception of words that could be used to describe any type of package ("pypi", "python", "module"...).

### Predicting keywords with TF-IDF

Now that we managed to rank the most important tokens of a project description using TF-IDF, we could try to see if the highest ranked tokens for each package are effective keywords predictors. (WIP)

## Predicting packages trove classifiers with Reinforcement learning

See: https://arxiv.org/pdf/1908.10419.pdf